# Distributional DQN
The final improvement to the DQN agent [1] is using distributions instead of simple average values for learning the q value function. This algorithm was presented by Bellemare et al. (2018) [2]. In their math heavy manuscript, the authors introduce the distributional Belman operator and show that it defines a contraction for the policy evaluation case. Bellemare et al. suggest that using distributions leads to a "significantly better behaved" [2] reinforcement learning and underline their theoretical findings with much better results on the Arcade Learning Environment [3]. Distributional DQN is also one of the improvements that seem to have one of the biggest impact in the Rainbow manuscript [4].

## Implementation
The distributional part of the algorithm is maybe the most influenced by the Deep Reinforcement Learning Hands-On book [5]. To get a good understanding of both, the distributional Bellman operator and how to use it in the DQN agent, I thoroughly studied the book's implementation as a starting point. I later implemented the distributional operator in a way that appears slightly more elegant to me, even though I still did not manage to get rid of the special treatment for the last step of an episode.  
Some of the functionality for distributions was implemented in a DistributionalNetHelper class. 
This way other neural network architectures can just inherit this functionality, even though at this point I only implemented DuelingDQN with distrbutions.

## Results

I start by comparing the Distributional DQN agent with two other agents. The different agents I compare are:

1. run024, which is the DQN agent with DDQN, Dueling DQN, n-step-reward and Prioritized Experience Replay. This agent so far had the most convincing results over the entire set of experiments.
2. run028, which uses all of the improvements mentioned above as wel as using Noisy Networks
3. run029 as run028 but using distributions instead of simple averages. 

The values for the support in the distributional agent were chosen similar to the manuscript [2]; the number of atoms was 51 with values between -10 and 10. These values are not optimal for some of the experiments, as will become evident later.

On the radar plot, it appears that the distributional agent has the worst performance of all three agents. However,  the barplot reveals that the DistributionalDQN agent shows good results on bandit, catch and catch_scale.
![Radar plot of te three different agents.](./figures/distributional_radar.png)

![Barplot of te three different agents.](./figures/distributional_barplot.png)

The problem that the DistributionalDQN agent faces is its use of a fixed support while the different experiments have very different reward scales. In the bandit experiment the final reward (and thus the q-value) varies between 0 and 1, while the cartpole experiment, for example, has q-values somewhere between 0 and 1001.

To investigate if using more appropriate vmin and vmax values yields better performance, I ran four of the experiments one more time with slightly different settings:
4. run030 uses the same settings as run029 but vmin = -1000 and vmax = 0, these settings were used for mountaincar and mountaincar_scale
5. run032 uses the same settings as run29 but vmin = 0, vmax = 1000, these settings were used for cartpole and cartpole_scale.

The results are shown below. It is apparent that the performance greatly improves when an appropriate scale is chosen. 
![Barplot of the three different agents.](./figures/distributional_barplot_tuned.png)

In the scaled versions of the experiments, one can observe that the fine-tuned agent (run031) only performed well for the scale of 1.0, while the first set of parameters was better for smaller scales. This shows how strongly the choice of the support for the distribution influences the results.

![Results for the mountain car experiment with different scales.](./figures/distributional_mountaincar_scale.png)

## Discussion

The results above show that the DistributionalDQN agent can learn good policies very well if an appropriate scale for the support is chosen. However, this is also the obvious problem of the approach in the form presented in [2]. 
When the support is not chosen in an appropriate way, using a simple average value is probably more robust than using distributions.

In [2], distributions were used to improve the convergence properties of the DQN algorithms. 
Using distributions has even more potential. One could use the distributions to improve action selection, for example in the case of multimodal distributions with very different (non-deterministic) rewards  and probabilities.
Consider one action that certainly gives a reward of +1 and another action that gives a reward of +100 with a probability of 1\%. Even though both actions yield an average reward of 1 they certainly have very different risk profiles, and this could be assesed when the whole distributional information is available.

## References

The figures here were produced by the analysis Jupyter Notebook from [the BSuite code repository](https://github.com/deepmind/bsuite) and [6].

[1] Mnih, Volodymyr, et al. Human-level control through deep reinforcement learning. Nature, 2015.  
[2] Bellemare, Marc G., Will Dabney, and Rémi Munos. "A distributional perspective on reinforcement learning." Proceedings of the 34th International Conference on Machine Learning-Volume 70. JMLR. org, 2017.  
[3] Bellemare, Marc G., et al. "The arcade learning environment: An evaluation platform for general agents." Journal of Artificial Intelligence Research 47 (2013): 253-279.  
[4] Hessel, Matteo, et al. Rainbow: Combining improvements in deep reinforcement learning. In: Thirty-Second AAAI Conference on Artificial Intelligence. 2018.  
[5] Lapan, Maxim. Deep Reinforcement Learning Hands-On, Packt Publishing Ltd, 2018.